In [12]:
#   |'''''''''''''╔╬╬╬╬╬╬╬╬   _____  _____      _____  _____      ___   __
#   |            ╔╬╬╬╬╬╬╬╬╬  |\   _ \  _  \    |\   _ \  _  \    |\  \|\  \
#   | ░░         ╬╬╬╬╬╬╬╬╬╬  \ \  \\__\ \  \   \ \  \\__\ \  \   \ \  \/  /|_
#    ░░░░        ╬╬╬╬╬╬╬╬╬╬   \ \  \|__| \  \   \ \  \|__| \  \   \ \   ___  \
#   ░░░░░╦╬╦    ╔╬╬╬╬╬╬╬╬╬╬    \ \  \   \ \  \   \ \  \   \ \  \   \ \  \ \   \
#  ░░░░░╬╬╬╬ ▓▓└╬╬╬╬╬╬╬╬╬╬╬     \ \__\   \ \__\   \ \__\   \ \__\   \ \__\ \___\
# ░░░░░╔╬╬╬ ▓▓▓  ╓╬╬╬╬╬╬╬╬╬      \|__|    \|__|    \|__|    \|__|    \|__| \|__|
# ░░░░░╠╬╬╬ ▓▓▓  └╬╬╬╬╬╬╬╬╬
#  ░░░░└╬╬╬╬ ▓▓   ╬╬╬╬╬╬╬╬╬  Lehrstuhl für Mensch-Maschine-Kommunikation
#  ░░░░░╙╬╬╬╩            ╬╬  Technische Universität München
#   ░░░░░░╚ '''''''''''''''  Author: Tobias Watzel
#    ░░░                     Copyright 2020
#

%matplotlib widget
import cv2
import sys
import io
import ipywidgets as widgets
from IPython.display import Image
from IPython.display import display, Markdown
import PIL.Image
import numpy as np 
import subprocess
import graphviz
from collections import Counter
import dill

# Versuchsbeschreibung:
In diesem Versuch soll das Datenkommpressionsverfahren nach Huffman veranschaulicht werden. Geben Sie hierzu ein längeres Wort oder einen kurzen Text in das Textfeld ein! Betrachten Sie anschließend die Huffman-Codierungstabelle und versuchen Sie, deren Entstehung anhand des Ihnen bekannten Huffman-Algorithmus nachzuvollziehen und zu überprüfen. Beachten Sie die erzielten Kompressionsraten und vergleichen Sie diese mit denen von professionellen Packern (rar,zip, etc.) bei Anwendung auf große Text-Dateien.

In [13]:
def draw_tree(tree, prefix = ''):    
    if isinstance(tree, str):            
        descr = 'N%s [label="%s: %s", fontcolor=blue, fontsize=18, width=1, shape=box];\n'%(prefix, tree.upper(), prefix)
    else: # Node description
        descr = 'N%s [label="%s"];\n'%(prefix, prefix)
        for child in tree.keys():
            descr += draw_tree(tree[child], prefix = prefix+child)
            descr += 'N%s -> N%s;\n'%(prefix,prefix+child)
    return descr

def convert_image(a, fmt='png'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    return f.getvalue()


with open('images/graph.dot','w') as f:
    f.write('digraph G {\n')
    f.write(draw_tree(tree))
    f.write('}') 
subprocess.call('dot -Tpng images/graph.dot -o images/graph.png', shell=True)
image1 = cv2.imread('images/graph.png')
    
image = widgets.Image(value=convert_image(image1), format='png', width=1500, height=1000)

display(image)


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x0c=\x00\x00\x02{\x08\x02\x00\x00\x00\xd5\x84\x9c!\x…

In [14]:
# inspired by https://stackoverflow.com/questions/11587044/how-can-i-create-a-tree-for-huffman-encoding-and-decoding/12656695

def assign_code(nodes, label, result, prefix = ''):    
    childs = nodes[label]     
    tree = {}
    if len(childs) == 2:
        tree['0'] = assign_code(nodes, childs[0], result, prefix+'0')
        tree['1'] = assign_code(nodes, childs[1], result, prefix+'1')     
        return tree
    else:
        result[label] = prefix
        return label

def huffman_code(_vals):    
    vals = _vals.copy()
    nodes = {}
    for n in vals.keys():
        nodes[n] = []

    while len(vals) > 1: 
        s_vals = sorted(vals.items(), key=lambda x:x[1]) 
        a1 = s_vals[0][0]
        a2 = s_vals[1][0]
        vals[a1+a2] = vals.pop(a1) + vals.pop(a2)
        nodes[a1+a2] = [a1, a2]        
    code = {}
    root = a1+a2
    tree = {}
    tree = assign_code(nodes, root, code)   # assignment of the code for the given binary tree      
    return code, tree

def change_input(change):
    pass

text = widgets.Text(value='Sprach- und Bildverarbeitung macht Spaß!', placeholder='Type something', description='Zu enkodierenden Text eingeben:', disabled=False, 
                    style = {'description_width': 'initial'}, layout = widgets.Layout(width = '700px'))
display(text)

vals = Counter(text.value)
code, tree = huffman_code(vals)

encoded = ''.join([code[t] for t in text.value])
print('Enkodierter Text:', encoded)

decoded = []
i = 0
while i < len(encoded): # decoding using the binary graph
    ch = encoded[i]  
    act = tree[ch]
    while not isinstance(act, str):
        i += 1
        ch = encoded[i]  
        act = act[ch]        
    decoded.append(act)          
    i += 1

print('Dekodierter Text:',''.join(decoded))

print('Speicherplatz Text: %i Byte (%i Bit)' % (len(text.value), len(text.value) * 8))
print('Speicherplatz enkodierter Text: %i Byte (%i Bit)' % (len(encoded) / 8, len(encoded)))

Text(value='Sprach- und Bildverarbeitung macht Spaß!', description='Zu enkodierenden Text eingeben:', layout=L…

Enkodierter Text: 1100111110101111011111100000111011100001001000111110011110100100000011100010101101111011011100100101010001100001001010011111010100110111111000001101110110011111011011010111000
Dekodierter Text: Sprach- und Bildverarbeitung macht Spaß!
Speicherplatz Text: 40 Byte (320 Bit)
Speicherplatz enkodierter Text: 21 Byte (175 Bit)


In [15]:
'''mine'''
def assign_code(nodes, label, result, prefix = ''):    
    childs = nodes[label]     
    tree = {}
    if len(childs) == 2:
        tree['0'] = assign_code(nodes, childs[0], result, prefix+'0')
        tree['1'] = assign_code(nodes, childs[1], result, prefix+'1')     
        return tree
    else:
        result[label] = prefix
        return label

def huffman_code(_vals):    
    vals = _vals.copy()
    nodes = {}
    for n in vals.keys():
        nodes[n] = []

    while len(vals) > 1: 
        s_vals = sorted(vals.items(), key=lambda x:x[1]) 
        a1 = s_vals[0][0]
        a2 = s_vals[1][0]
        vals[a1+a2] = vals.pop(a1) + vals.pop(a2)
        nodes[a1+a2] = [a1, a2]        
    code = {}
    root = a1+a2
    tree = {}
    tree = assign_code(nodes, root, code)   # assignment of the code for the given binary tree      
    return code, tree

def change_input(change):
    pass

text = widgets.Text(value='Franz jagt im komplett verwahrlosten Taxi quer durch Bayern', placeholder='Type something', description='Zu enkodierenden Text eingeben:', disabled=False, 
                    style = {'description_width': 'initial'}, layout = widgets.Layout(width = '700px'))
display(text)

vals = Counter(text.value)
code, tree = huffman_code(vals)

encoded = ''.join([code[t] for t in text.value])
print('Enkodierter Text:', encoded)

decoded = []
i = 0
while i < len(encoded): # decoding using the binary graph
    ch = encoded[i]  
    act = tree[ch]
    while not isinstance(act, str):
        i += 1
        ch = encoded[i]  
        act = act[ch]        
    decoded.append(act)          
    i += 1

print('Dekodierter Text:',''.join(decoded))

print('Speicherplatz Text: %i Byte (%i Bit)' % (len(text.value), len(text.value) * 8))
print('Speicherplatz enkodierter Text: %i Byte (%i Bit)' % (len(encoded) / 8, len(encoded)))

Text(value='Franz jagt im komplett verwahrlosten Taxi quer durch Bayern', description='Zu enkodierenden Text e…

Enkodierter Text: 101100001111101001011010111011101111101111010101110000100010111100001001010001110001100110000101010101111001000000111001111111010000110011100101101000101000010001111010111111101101000001111011110101000001011111000101010011110011010001111101011111110110000010100
Dekodierter Text: Franz jagt im komplett verwahrlosten Taxi quer durch Bayern
Speicherplatz Text: 59 Byte (472 Bit)
Speicherplatz enkodierter Text: 32 Byte (261 Bit)


### 1. Auf wieviel Prozent reduziert sich der Speicherbedarf bei Huffman-Kodierung des Wortes "Mississippi"?

In [16]:
save_mississippi = 'save/size_mississippi.dill'

size_mississippi = ''
# try to load precicion value
try: 
    with open(save_mississippi, 'rb') as fp:
        size_mississippi = dill.load(fp)
        if size_mississippi == '':
            size_mississippi = 0.0
except:
    pass

text_size_mississippi = widgets.Text(value=str(size_mississippi),
                              placeholder='',
                              description='',
                              disabled=False,
                              continuous_update=True)

def callback_size_mississippi(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # save when press enter
        with open(save_mississippi, 'wb') as fp:
            dill.dump(change['new'], fp)


text_size_mississippi.observe(callback_size_mississippi)

label = widgets.Label(value = '% gerundet ohne Nachkommastellen ')
display(widgets.HBox([text_size_mississippi, label]))

In [17]:
# Autograding answer, please ignore


### 2. Auf wieviel Prozent reduziert sich der Speicherbedarf bei Huffman-Kodierung von "Franz jagt im komplett verwahrlosten Taxi quer durch Bayern" ?

In [18]:
save_franz = 'save/size_franz.dill'

size_franz = ''
# try to load precicion value
try: 
    with open(save_franz, 'rb') as fp:
        size_franz = dill.load(fp)
        if size_franz == '':
            size_franz = 0.0
except:
    pass

text_size_franz = widgets.Text(value=str(size_franz),
                              placeholder='',
                              description='',
                              disabled=False,
                              continuous_update=True)

def callback_size_franz(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # save when press enter
        with open(save_franz, 'wb') as fp:
            dill.dump(change['new'], fp)


text_size_franz.observe(callback_size_franz)

label = widgets.Label(value = '% gerundet ohne Nachkommastellen ')
display(widgets.HBox([text_size_franz, label]))

In [19]:
# Autograding answer, please ignore
